<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Calculate-number-of-clusters-in-rewired" data-toc-modified-id="Calculate-number-of-clusters-in-rewired-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Calculate number of clusters in rewired</a></span></li></ul></div>

In [2]:
%config Completer.use_jedi = False

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from enm.Enm import Enm, rewire_network
from enm.utils import *
import pickle
import random
random.seed(4812)        # or any integer
np.random.seed(4813)

In [4]:
strain_ids =pd.read_csv(snakemake.input.strain_ids)

In [5]:
e = Enm('rew')

In [6]:
e.read_network(snakemake.input.pcc_all,sep=',')

In [7]:
gc_rew = rewire_network(e.graph_gc)
e_rew = Enm('rewired')
e_rew.G=gc_rew
e_rew.giant_component()
e_rew.gnm_analysis()
e_rew.df = pd.merge(e_rew.df,strain_ids, left_on='orf_name',right_on='gene1')
e_rew.get_sensor_effector(True)

In [9]:
snakemake.input

In [11]:
goea, geneid2name,a = create_goea(gaf=snakemake.input.gaf, 
                                  obo_fname=snakemake.input.obo,
                                  background=snakemake.input.background_file,
                               sgd_info_tab=snakemake.input.sgd_info)

In [12]:
e_rew.analyze_components_biology(goea, geneid2name,True)
e_rew.analyze_components_biology(goea, geneid2name,False)

In [13]:
e_rew.get_sensor_effector()

In [14]:
sensors_pcc = e_rew.sensors_df

In [15]:
pos = e_rew.graph_gc.nodes('pos')

In [16]:
from matplotlib.lines import Line2D

fig, ax = plt.subplots(figsize=(9,9))
legend_elements = [    ]

e_rew.plot_network_spring(ax=ax,
                          node_size=1,
                          node_color='black',

                         edge_color='black',savefig=False)

nx.draw_networkx_nodes(nx.induced_subgraph(e_rew.graph_gc, sensors_pcc.orf_name.tolist()),
                       pos=pos, 
                       node_color='black', alpha=1, node_shape='^')

ax.set_facecolor('white')
# legend_elements.append(
#         Line2D([0], [0], marker='^', color='black', label=f'No GO Enrichment',
#                               markerfacecolor='black', markersize=30, linestyle="None")
#     )
legend_elements.extend(
    [Line2D([0], [0], marker='^', color='black', label='Sensors',
                              markerfacecolor='black', markersize=10, linestyle="None"),
     Line2D([0], [0], marker='o', color='black', label='Non-sensor Genes',
                              markerfacecolor='black', markersize=10, linestyle="None"),
#                    Line2D([0], [0], marker='o', color='black', label='Effectors',
#                               markerfacecolor='black', markersize=10, linestyle="None"),
                   
                       Line2D([0], [0], marker='o', color='black', label= 'High functional similarity',
                              markerfacecolor='black', markersize=0, linestyle="-", alpha=0.5, lw=10),
                   Line2D([0], [0], marker='o', color='red', label= 'Sensor-Sensor edges',
                              markerfacecolor='#018571', markersize=0, linestyle="-",lw=10)
                   #Line2D([0], [0], marker='o', color='blue', label= 'Effector-Effector edges',
    #                          markerfacecolor='#a6611a', markersize=0, linestyle="-")
    ]
)
lgd = ax.legend(handles=legend_elements, fontsize=22,
                loc='center right', bbox_to_anchor=(1.8, 0.5),ncol=1,
               frameon=False)
nx.draw_networkx_edges(nx.induced_subgraph(e_rew.graph_gc, sensors_pcc.orf_name.tolist()),pos=pos, edge_color='red', alpha=0.5)
ax.axis('off')
if snakemake.params['save']:
    plt.savefig(f'reports/figures/paper_figures_supp/figs1.png',bbox_inches='tight',dpi=150)

# Calculate number of clusters in rewired

In [17]:
with open(snakemake.input.pickle_file_name,'rb') as f:
    e_pcc = pickle.load(f)

In [18]:
%%capture
e_pcc.simulate_rewire(sim_num = snakemake.params['sim_num'])

In [19]:
for i in e_pcc.e_list:
    i.df = pd.merge(i.df , strain_ids , left_on = 'orf_name', right_on='gene1')
    i.get_sensor_effector()
    i.analyze_components_biology(goea, geneid2name)

In [20]:
def save_rewired_data(e_pcc, idx, path):
    e = e_pcc.e_list[idx]
    df_rew = e.df
    sensors_df_rew = e.sensors_df
    effectors_df_rew = e.effectors_df
    g = e.graph_gc
    MYDIR = (f"{path}/{idx}")
    CHECK_FOLDER = os.path.isdir(MYDIR)

    # If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        os.makedirs(MYDIR)
        print("created folder : ", MYDIR)

    else:
        print(MYDIR, "folder already exists.")
    df_rew.to_csv(f"{path}/{idx}/df_rew_{idx}.csv",index=False)
    sensors_df_rew.to_csv(f"{path}/{idx}/sensors_df_rew_{idx}.csv",index=False)
    effectors_df_rew.to_csv(f"{path}/{idx}/effectors_df_rew_{idx}.csv",index=False)
    nx.write_edgelist(g, f"{path}/{idx}/g_rew_{idx}.edgelist.gz")

In [21]:
CHECK_FOLDER = os.path.isdir(snakemake.output.rewired_data_folder)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(snakemake.output.rewired_data_folder)

In [22]:
[save_rewired_data(e_pcc,i,snakemake.output.rewired_data_folder) for i in range(snakemake.params['sim_num'])]

In [23]:
import glob

sensors_fls = glob.glob(f'{snakemake.output.rewired_data_folder}/sensors_df_rew*')

In [25]:
sensors_df_rew = pd.concat([pd.read_csv(f'{snakemake.output.rewired_data_folder}/{idx}/sensors_df_rew_{idx}.csv') for idx in range(snakemake.params['sim_num'])],keys=range(snakemake.params['sim_num'])).reset_index(level=0)

In [26]:
res = sensors_df_rew.groupby('level_0')['sensor_cluster'].nunique()
res2 = sensors_df_rew.groupby('level_0')['go_group'].nunique()

In [27]:
#res = [i.sensors_df.dropna(subset=['sensor_cluster']).sensor_cluster.nunique() for i in e_pcc.e_list]
#res2 = [i.sensors_df.dropna(subset=['go_group']).go_group.nunique() for i in e_pcc.e_list]

In [28]:
idxx = np.argwhere(np.array(res)>0).reshape(1,-1)[0]
idxx2 = np.argwhere(np.array(res2)>0).reshape(1,-1)[0]

There are 68 rewired networks (out of 500) with a sensor cluster

In [29]:
len(idxx)

This corresponds to 13.6% of rewired networks

In [30]:
try:
    print(len(idxx)/len(res))
except Exception as e:
    print(len(idxx))

While 25% of these are GO enriched, that corresponds to only 3.4% of 500 cases

In [31]:
try:
    print(len(idxx2)/len(res2))
except Exception as e:
    print(len(idxx))

In [32]:
try:
    print(len(idxx2)/len(idxx))
except Exception as e:
    print(len(idxx))

In [65]:
rewired_nw_list = [nx.read_edgelist(f'{snakemake.output.rewired_data_folder}/{idx}/g_rew_{idx}.edgelist.gz') for idx in range(snakemake.params['sim_num'])]

In [74]:
def plot_sensors(rew_list,idx ,sensors_df_list):
    g =rew_list[idx]
    sub_orfs =  sensors_df_list.loc[sensors_df_list.level_0==idx].dropna(subset=['sensor_cluster']).orf_name.tolist()
    #g = e.graph_gc
    induced_g = nx.induced_subgraph(g,sub_orfs)
    sub_nw = get_subnetwork(g, sub_orfs, radius= 1)
    pos_sub = nx.spring_layout(sub_nw)
    fig, ax_ = plt.subplots() 
    nx.draw_networkx_nodes(sub_nw,ax=ax_, pos = pos_sub, node_color = ['none' if i in sub_orfs else 'k' for i in sub_nw.nodes])
    nx.draw_networkx_nodes(nx.induced_subgraph(sub_nw, sub_orfs), pos=pos_sub, node_shape='^', node_color='black')
    nx.draw_networkx_edges(sub_nw,ax=ax_, pos = pos_sub)
    nx.draw_networkx_edges(nx.induced_subgraph(sub_nw, sub_orfs), pos=pos_sub, edge_color='red')
    plt.show()
    #nx.draw_networkx(sub_nw)

In [131]:
def find_antenna_sensors(rew_list,idx ,sensors_df_list):
    g =rew_list[idx]
    sub_orfs =  sensors_df_list.loc[sensors_df_list.level_0==idx].dropna(subset=['sensor_cluster']).orf_name.tolist()
    #g = e.graph_gc
    induced_g = nx.induced_subgraph(g,sub_orfs)
    sub_nw = get_subnetwork(g, sub_orfs, radius= 1)
    all_nodes = sub_nw.nodes
    sensor_nodes = sub_orfs
    num_nonsensor = len(np.setdiff1d(all_nodes, sensor_nodes)) / len(list(nx.connected_components(sub_nw)))
    #print(num_nonsensor)
    return num_nonsensor==1, len(np.setdiff1d(all_nodes, sensor_nodes))

In [84]:
for i in idxx:
    print(i)
    plot_sensors(rewired_nw_list,i,sensors_df_rew)

%96 of rewired networks with a sensor cluster have that sensor cluster as antenna

In [33]:
fig, ax =plt.subplots(figsize=(5,5))
ax.hist(res,3,color='navajowhite')
ax.set_xlabel('Number of sensor clusters')
ax.set_ylabel('Count')
ax.set_title('Number of sensor clusters\nat 500 rewired networks\ncompared to real network')
ax.axvline(9,c='darkblue',linestyle='-.')
plt.legend(handles = [
    Line2D([0],[0],color='navajowhite',linewidth=10,label='Rewired'),
    Line2D([0],[0],color='darkblue',linestyle='-.', label='Real')
], loc='upper center')
#if snakemake.params['save']:
 #   fig.savefig('reports/figures/paper_figures_supp/rewired_sensor_count.png', bbox_inches='tight',dpi=150)

In [57]:
fig, ax =plt.subplots(figsize=(5,5))
ax.hist(sensors_df_rew.groupby('level_0')['go_group'].nunique(),3,color='navajowhite')
ax.set_xlabel('Number of GO enriched sensor clusters')
ax.set_ylabel('Count')
ax.set_title('Number of GO enriched\nsensor clusters\nat 500 rewired networks\ncompared to real network')
ax.axvline(5,c='darkblue',linestyle='-.')
plt.legend(handles = [
    Line2D([0],[0],color='navajowhite',linewidth=10,label='Rewired'),
    Line2D([0],[0],color='darkblue',linestyle='-.', label='Real')
], loc='upper center')
if snakemake.params['save']:
    fig.savefig('reports/figures/paper_figures_supp/rewired_go_sensor_count.png', bbox_inches='tight',dpi=150)

In [176]:
fig, ax =plt.subplots(figsize=(5,5))
ax.hist(res,3,color='navajowhite')
ax.set_xlabel('Number of sensor clusters')
ax.set_ylabel('Count')
ax.set_title('Number of sensor clusters\nat 100 rewired networks\ncompared to real network')
ax.axvline(9,c='darkblue',linestyle='-.')
plt.legend(handles = [
    Line2D([0],[0],color='navajowhite',linewidth=10,label='Rewired'),
    Line2D([0],[0],color='darkblue',linestyle='-.', label='Real')
], loc='upper center')
if snakemake.params['save']:
    fig.savefig('reports/figures/paper_figures_supp/rewired_sensor_count.png', bbox_inches='tight',dpi=150)

In [130]:
try:
    print(sum([find_antenna_sensors(rewired_nw_list,i,sensors_df_rew) for i in idxx]) / len(idxx))
    fig, ax =plt.subplots(figsize=(5,5))
    ax.hist([j for i,j in rew_antenna if i==True],3,color='navajowhite')
    ax.axvline(6,c='darkblue',linestyle='-.')
    ax.set_xlabel('Number of antenna motifs')
    ax.set_ylabel('Count')
    ax.set_title('Number of antenna motifs\nat 68 rewired networks with sensor clusters\ncompared to real network')
    plt.legend(handles = [
        Line2D([0],[0],color='navajowhite',linewidth=10,label='Rewired'),
        Line2D([0],[0],color='darkblue',linestyle='-.', label='Real')
    ], loc='upper center')
    if snakemake.params['save']:
        fig.savefig('reports/figures/paper_figures_supp/rewired_antenna_sensor_count.png', bbox_inches='tight',dpi=150)
except Exception as e:
    print('no clusters were found')

In [132]:
rew_antenna = [find_antenna_sensors(rewired_nw_list,i,sensors_df_rew) for i in idxx]